# Big Data - Datastorm

## Initializing Spark

In [1]:
# pip install pyspark
from pyspark.sql import SparkSession, functions, types, Column, Row

spark = (SparkSession
  .builder
  .appName("Datastorm")
  .getOrCreate())
sc = spark.sparkContext

22/11/16 22:24:11 WARN Utils: Your hostname, Karishmas-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.145 instead (on interface en0)
22/11/16 22:24:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/16 22:24:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Data Preparation

## ETL - Phase 1 - Extract 

### 1. Loading The Different Sets of Data

#### 1.1 GHCN Daily Data

Loading the data using pyspark and specifying the required schema

In [2]:
ghcn_data = 'data/ghcn'

observation_schema = types.StructType([
types.StructField('station', types.StringType()),
types.StructField('date', types.StringType()),
types.StructField('observation', types.StringType()),
types.StructField('value', types.IntegerType()),
types.StructField('mflag', types.StringType()),
types.StructField('qflag', types.StringType()),
types.StructField('sflag', types.StringType()),
types.StructField('obstime', types.StringType()),
])


ghcn_df = spark.read.csv(ghcn_data, schema = observation_schema)


ghcn_df.show()

+-----------+--------+-----------+-----+-----+-----+-----+-------+
|    station|    date|observation|value|mflag|qflag|sflag|obstime|
+-----------+--------+-----------+-----+-----+-----+-----+-------+
|US1NHST0018|20100101|       PRCP|   15| null| null|    N|   null|
|US1NHST0018|20100101|       SNOW|   20| null| null|    N|   null|
|US1NHST0018|20100101|       WESF|   13| null| null|    N|   null|
|US1NJSS0023|20100101|       PRCP|   76| null| null|    N|   null|
|US1NJSS0023|20100101|       SNOW|   81| null| null|    N|   null|
|US1NJSS0023|20100101|       SNWD|   76| null| null|    N|   null|
|US1NJSS0023|20100101|       WESD|   76| null| null|    N|   null|
|US1NJSS0023|20100101|       WESF|   76| null| null|    N|   null|
|ASN00015643|20100101|       TMAX|  414| null| null|    a|   null|
|ASN00015643|20100101|       TMIN|  254| null| null|    a|   null|
|ASN00015643|20100101|       PRCP|    0| null| null|    a|   null|
|US1NCTR0002|20100101|       PRCP|    3| null| null|    N|   n

#### 1.2 GHCN Station Data

Loading the data using pyspark and specifying the required schema

In [3]:
import re

In [4]:
def stations_schema():
    return types.StructType([
        types.StructField("Station ID", types.StringType()),
        types.StructField("Latitude", types.FloatType()),
        types.StructField("Longitude", types.FloatType()),
        types.StructField("Elevation", types.FloatType()),
        types.StructField("State", types.StringType()),
        types.StructField("Station Name", types.StringType()),
        #types.StructField("gsn_flag", types.StringType()),
        #types.StructField("crn_flag", types.StringType()),
        #types.StructField("wmo_id", types.StringType()),
    ])

In [5]:
def parse_line(line):
    id = '(\S+)'
    latitude = '([-+]?(?:\d*\.\d+|\d+))'
    longitude = '([-+]?(?:\d*\.\d+|\d+))'
    elevation = '([-+]?(?:\d*\.\d+|\d+))'
    state = '([-a-zA-Z0-9_][-a-zA-Z0-9_])'
    name = '((\S+\s)+)'
    delimiter = '\s+'
    any = ".*"
    line_re = re.compile(r'^'+ id + delimiter + latitude + delimiter + longitude + delimiter + elevation + delimiter + state + delimiter + name + any + '$')
    splitted_line = re.match(line_re, line)
    return Row(splitted_line.group(1), float(splitted_line.group(2)), float(splitted_line.group(3)), float(splitted_line.group(4)), splitted_line.group(5), splitted_line.group(6))

In [20]:
stations_input = sc.textFile("data/ghcnd-stations.txt")
formatted_lines = stations_input.filter(lambda line: line.startswith("CA")).map(parse_line)
cleaned_stations = spark.createDataFrame(data=formatted_lines, schema = stations_schema())
cleaned_stations.show()

canada_stations_dataframes = cleaned_stations
canada_stations_dataframes.show()

+-----------+--------+---------+---------+-----+--------------------+
| Station ID|Latitude|Longitude|Elevation|State|        Station Name|
+-----------+--------+---------+---------+-----+--------------------+
|CA001010066| 48.8667|-123.2833|      4.0|   BC|        ACTIVE PASS |
|CA001010235|    48.4|-123.4833|     17.0|   BC|        ALBERT HEAD |
|CA001010595| 48.5833|-123.5167|     85.0|   BC|BAMBERTON OCEAN C...|
|CA001010720|    48.5|   -124.0|    351.0|   BC|         BEAR CREEK |
|CA001010774|    48.5|  -123.35|     61.0|   BC|        BEAVER LAKE |
|CA001010780| 48.3333|-123.6333|     12.0|   BC|         BECHER BAY |
|CA001010960|    48.6|-123.4667|     38.0|   BC|    BRENTWOOD BAY 2 |
|CA001010961| 48.5667|  -123.45|     31.0|   BC|BRENTWOOD CLARKE ...|
|CA001010965| 48.5667|-123.4333|     91.0|   BC|BRENTWOOD W SAANI...|
|CA001011467| 48.5833|-123.4167|     53.0|   BC|CENTRAL SAANICH V...|
|CA0010114F6| 48.5667|   -123.4|     38.0|   BC|CENTRAL SAANICH I...|
|CA0010114FF|   48.5

## ETL - Phase 2 - Transform

### GHCN Daily Data

#### 2.1 Extracting Only Canada Data Using Station Name

In [21]:
canada_weather_data = ghcn_df.filter(ghcn_df.station.startswith('CA'))
canada_weather_data.show()

+-----------+--------+-----------+-----+-----+-----+-----+-------+
|    station|    date|observation|value|mflag|qflag|sflag|obstime|
+-----------+--------+-----------+-----+-----+-----+-----+-------+
|CA007020860|20100101|       TMAX|    0| null| null|    C|   null|
|CA007020860|20100101|       TMIN|  -50| null| null|    C|   null|
|CA007020860|20100101|       PRCP|   30| null| null|    C|   null|
|CA007020860|20100101|       SNOW|   30| null| null|    C|   null|
|CA007020860|20100101|       SNWD|  120| null| null|    C|   null|
|CA005030984|20100101|       TMAX| -234| null| null|    C|   null|
|CA005030984|20100101|       TMIN| -276| null| null|    C|   null|
|CA005030984|20100101|       PRCP|   13| null| null|    C|   null|
|CA005030984|20100101|       SNWD|  380| null| null|    C|   null|
|CA005023225|20100101|       PRCP|    0| null| null|    C|   null|
|CA005023225|20100101|       SNOW|    0| null| null|    C|   null|
|CA005023225|20100101|       SNWD|  200| null| null|    C|   n

####     2.2 Checking for & Removing Unnecessary Columns

mflag indicates measurement flags that are not required for this study.

sflag indicates the source of the data collected which is also not required for this study. 

obstime indicates the time at which the data was recorded and can be dropped as well.

In [22]:
canada_weather_data = canada_weather_data.drop("sflag", "mflag", "obstime")
canada_weather_data.show()

+-----------+--------+-----------+-----+-----+
|    station|    date|observation|value|qflag|
+-----------+--------+-----------+-----+-----+
|CA007020860|20100101|       TMAX|    0| null|
|CA007020860|20100101|       TMIN|  -50| null|
|CA007020860|20100101|       PRCP|   30| null|
|CA007020860|20100101|       SNOW|   30| null|
|CA007020860|20100101|       SNWD|  120| null|
|CA005030984|20100101|       TMAX| -234| null|
|CA005030984|20100101|       TMIN| -276| null|
|CA005030984|20100101|       PRCP|   13| null|
|CA005030984|20100101|       SNWD|  380| null|
|CA005023225|20100101|       PRCP|    0| null|
|CA005023225|20100101|       SNOW|    0| null|
|CA005023225|20100101|       SNWD|  200| null|
|CA003076680|20100101|       TMAX| -168| null|
|CA003076680|20100101|       TMIN| -222| null|
|CA003076680|20100101|       PRCP|    4| null|
|CA00306A009|20100101|       TMAX| -152| null|
|CA00306A009|20100101|       TMIN| -243| null|
|CA00306A009|20100101|       PRCP|    0| null|
|CA00306A009|

#### 2.3 Checking for & Eliminating Duplicates/Insufficienes in Data

qflag being null indicates that the specific data entry did not pass all of the necessary quality checks. In order to ensure that the data we use is of the highest quality, we eliminate any rows that do not satisfy this condition.

In [23]:
canada_weather_data = canada_weather_data.filter(canada_weather_data['qflag'].isNull())
canada_weather_data.show()


+-----------+--------+-----------+-----+-----+
|    station|    date|observation|value|qflag|
+-----------+--------+-----------+-----+-----+
|CA007020860|20100101|       TMAX|    0| null|
|CA007020860|20100101|       TMIN|  -50| null|
|CA007020860|20100101|       PRCP|   30| null|
|CA007020860|20100101|       SNOW|   30| null|
|CA007020860|20100101|       SNWD|  120| null|
|CA005030984|20100101|       TMAX| -234| null|
|CA005030984|20100101|       TMIN| -276| null|
|CA005030984|20100101|       PRCP|   13| null|
|CA005030984|20100101|       SNWD|  380| null|
|CA005023225|20100101|       PRCP|    0| null|
|CA005023225|20100101|       SNOW|    0| null|
|CA005023225|20100101|       SNWD|  200| null|
|CA003076680|20100101|       TMAX| -168| null|
|CA003076680|20100101|       TMIN| -222| null|
|CA003076680|20100101|       PRCP|    4| null|
|CA00306A009|20100101|       TMAX| -152| null|
|CA00306A009|20100101|       TMIN| -243| null|
|CA00306A009|20100101|       PRCP|    0| null|
|CA00306A009|

In [24]:
canada_weather_data = canada_weather_data.drop('qflag')

In [25]:
canada_weather_data = canada_weather_data.filter(canada_weather_data['date'].like('19%') | canada_weather_data['date'].like('20%'))
canada_weather_data.show()

+-----------+--------+-----------+-----+
|    station|    date|observation|value|
+-----------+--------+-----------+-----+
|CA007020860|20100101|       TMAX|    0|
|CA007020860|20100101|       TMIN|  -50|
|CA007020860|20100101|       PRCP|   30|
|CA007020860|20100101|       SNOW|   30|
|CA007020860|20100101|       SNWD|  120|
|CA005030984|20100101|       TMAX| -234|
|CA005030984|20100101|       TMIN| -276|
|CA005030984|20100101|       PRCP|   13|
|CA005030984|20100101|       SNWD|  380|
|CA005023225|20100101|       PRCP|    0|
|CA005023225|20100101|       SNOW|    0|
|CA005023225|20100101|       SNWD|  200|
|CA003076680|20100101|       TMAX| -168|
|CA003076680|20100101|       TMIN| -222|
|CA003076680|20100101|       PRCP|    4|
|CA00306A009|20100101|       TMAX| -152|
|CA00306A009|20100101|       TMIN| -243|
|CA00306A009|20100101|       PRCP|    0|
|CA00306A009|20100101|       SNOW|    0|
|CA00306A009|20100101|       SNWD|  300|
+-----------+--------+-----------+-----+
only showing top

### GHCN Stations Data

#### 2.5 Extracting Only Canada Data Using Station Name 

#### 2.6 Checking for & Eliminating Duplicates/Insufficienes in Data

In [26]:
print("Before Dropping Duplicates: ")
print(canada_stations_dataframes.count())
canada_stations_dataframes = canada_stations_dataframes.dropDuplicates()
print("After Dropping Duplicates: ")
print(canada_stations_dataframes.count())

Before Dropping Duplicates: 
9127
After Dropping Duplicates: 


9127


## ETL - Phase 3 - Load 

### 3.1 Writing back files as required

In [27]:
canada_stations_dataframes.coalesce(1).write.option("header",True) \
        .mode("overwrite") \
        .csv("stations")


In [28]:
canada_stations_dataframes.coalesce(1).write.option("header",True) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("stations by state")


## Data Analysis

## Research & Hypothesis

### Columns of Interest: 
    . PRCP = Precipitation (mm or inches as per user preference, inches to hundredths on Daily Form pdf file)
    . TAVG = Average Temperature
    . TMAX = Maximum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on Daily Form pdf file
    . TMIN = Minimum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on Daily Form pdf file
    . ACMC = Average cloudiness midnight to midnight from 30-second ceilometer data (percent)
    . AWND = Average daily wind speed (meters per second or miles per hour as per user preference)
    . TSUN = Daily total sunshine (minutes)
    . WT** = Weather Type where ** has one of the following values:
        . WT03 = Thunder
        . WT05 = Hail (may include small hail)
        . WT10 = Tornado, Water Spout, Funnel Cloud
        . WT11 = High or damaging winds
        . WT12 = Blowing Spray
        . WT13 = Mist
        . WT14 = Drizzle 
        . WT15 = Freezing drizzle 
        . WT16 = Rain (may include freezing rain, drizzle, and freezing drizzle)
        . WT17 = Freezing Rain
    . WVxx = Weather in the Vicinity where “xx” has one of the following values
        . WV03 = Thunder
        . WV20 = Rain or snow shower

In [29]:
columns_of_interest = ["PRCP","TMAX", "TAVG", "TMIN","ACMC","AWND", "TSUN","WT03","WT05","WT10","WT11","WT12","WT13","WT14","WT15","WT16","WT17","WV03","WV20"]
canada_weather_data =  canada_weather_data.filter(canada_weather_data.observation.isin(columns_of_interest))
canada_weather_data.show()
canada_weather_counts = canada_weather_data.groupBy('observation').count()
canada_weather_counts.show()

# print(canada_weather_data.count())

+-----------+--------+-----------+-----+
|    station|    date|observation|value|
+-----------+--------+-----------+-----+
|CA007020860|20100101|       TMAX|    0|
|CA007020860|20100101|       TMIN|  -50|
|CA007020860|20100101|       PRCP|   30|
|CA005030984|20100101|       TMAX| -234|
|CA005030984|20100101|       TMIN| -276|
|CA005030984|20100101|       PRCP|   13|
|CA005023225|20100101|       PRCP|    0|
|CA003076680|20100101|       TMAX| -168|
|CA003076680|20100101|       TMIN| -222|
|CA003076680|20100101|       PRCP|    4|
|CA00306A009|20100101|       TMAX| -152|
|CA00306A009|20100101|       TMIN| -243|
|CA00306A009|20100101|       PRCP|    0|
|CA003072151|20100101|       TMAX| -190|
|CA003072151|20100101|       TMIN| -294|
|CA003031094|20100101|       TMAX|  -83|
|CA003031094|20100101|       TMIN| -231|
|CA003031094|20100101|       PRCP|    0|
|CA002100719|20100101|       TMAX| -225|
|CA002100719|20100101|       TMIN| -295|
+-----------+--------+-----------+-----+
only showing top

+-----------+--------+
|observation|   count|
+-----------+--------+
|       TMIN|47601029|
|       TMAX|47592990|
|       PRCP|56553497|
|       WT16|    8813|
|       WT03|     251|
|       WT05|      43|
+-----------+--------+



Total Number of Rows In Canada Weather Data = 13753314\
Total Number of Rows In Canada Stations = 9217

### Joining Weather Data with Station Data

In [30]:
canada_weather_data = canada_weather_data.join(functions.broadcast(canada_stations_dataframes),canada_weather_data['station'] == canada_stations_dataframes['Station ID']).drop('Station ID')
canada_weather_data.show()

+-----------+--------+-----------+-----+--------+---------+---------+-----+--------------------+
|    station|    date|observation|value|Latitude|Longitude|Elevation|State|        Station Name|
+-----------+--------+-----------+-----+--------+---------+---------+-----+--------------------+
|CA007020860|20100101|       TMAX|    0| 45.4833|   -71.95|    130.0|   QC|      BROMPTONVILLE |
|CA007020860|20100101|       TMIN|  -50| 45.4833|   -71.95|    130.0|   QC|      BROMPTONVILLE |
|CA007020860|20100101|       PRCP|   30| 45.4833|   -71.95|    130.0|   QC|      BROMPTONVILLE |
|CA005030984|20100101|       TMAX| -234| 52.8167| -97.6167|    223.0|   MB|GEORGE ISLAND (AUT) |
|CA005030984|20100101|       TMIN| -276| 52.8167| -97.6167|    223.0|   MB|GEORGE ISLAND (AUT) |
|CA005030984|20100101|       PRCP|   13| 52.8167| -97.6167|    223.0|   MB|GEORGE ISLAND (AUT) |
|CA005023225|20100101|       PRCP|    0|   49.85| -97.2667|    238.0|   MB|WINNIPEG CHARLESW...|
|CA003076680|20100101|       T

In [ ]:
canada_weather_data = canada_weather_data.

### Partitioning Of Data Into Required Folders For Ease Of Analysis

In [31]:
canada_weather_data.write.option("header",True) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("states")

In [32]:
canada_weather_data.write.option("header",True) \
        .partitionBy("observation") \
        .mode("overwrite") \
        .csv("observation")

In [33]:
canada_weather_data.write.option("header",True) \
        .partitionBy("state", "observation") \
        .mode("overwrite") \
        .csv("observations by state")

22/11/17 01:12:37 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 698076 ms exceeds timeout 120000 ms
22/11/17 01:12:37 WARN SparkContext: Killing executors is not supported by current scheduler.
